### In this section, I will chunk the cleaned medicine data 
    1. So that embedding model(BERTModel) can process it properly.
    2. So that we can retrieve relevant docs efficiently based on the query.

### Analysis:
1. As per the previous stage, we saw each medicine has multiple sub sections. Most of sub sections' contents have less than 500 words.
2. The embedding model I choose is sentence-transformers/embeddinggemma-300m-medical. It can handle 1024 tokens at a time.
![](../screenshots/length_distribution.png "")
3. I will embedding hypothentical questions instead of the contents. 

#### Conclusion: I don't need to split the content that small. I will keep the original content as many as possible. So I will just split the content which's length is larger than 500. 


In [1]:
import json
import uuid
import os
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# This function will load the json file to json object
def load_json_list(path: str):    
    with open(path, mode = "r", encoding="utf-8") as f:
        return json.load(f)

In [3]:
workspace_base_path = os.getcwd()
parent_path = os.path.dirname(workspace_base_path)
dataset_path = os.path.join(parent_path, "data-pipeline" ,"datasets", "cleaned_medicine_data.json") 
print(dataset_path)

c:\Users\Montr\AI_Projects\MediPal\data-pipeline\datasets\cleaned_medicine_data.json


In [5]:
data = load_json_list(dataset_path)

In [6]:
data[:1]

[{'id': 1,
  'drug_name': 'Phenylephrine',
  'pronunciation': "pronounced as (fen il ef' rin)",
  'url': 'https://medlineplus.gov/druginfo/meds/a606008.html',
  'subtitles': [{'title': 'why is Phenylephrine prescribed?',
    'content': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'},
   {'title': 'how should Phenylephrine be used?',
    'content': "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not under

In [ ]:
""" 
split text by a fixed size with overlap.
It is very important to know what the max length of the input chunk. The choosen embedding model document will tell you so.
In this case, we choose sentence-transformers/embeddinggemma-300m-medical. It can handle 1024 tokens at a time. 
"""
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)

In [8]:
approch_orig_data = []
for d in data:    
    for t in d["subtitles"]:   
        id_key = str(uuid.uuid4()) 
        docs_list = [] 
        chunked_docs = text_splitter.create_documents([t["content"]])
        for cd in chunked_docs:
            docs_list.append(cd.page_content)        
        approch_orig_data.append({"doc_id" : id_key, "questions" : [t["title"]], "docs" : docs_list,"original_doc":t["content"]})

In [9]:
approch_orig_data[:3]

[{'doc_id': 'b2bf17e4-42dc-4647-bde3-f6314b67c650',
  'questions': ['why is Phenylephrine prescribed?'],
  'docs': ['phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'],
  'original_doc': 'phenylephrine is used to relieve nasal discomfort caused by colds, allergies, and hay fever. it is also used to relieve sinus congestion and pressure. phenylephrine will relieve symptoms but will not treat the cause of the symptoms or speed recovery. phenylephrine is in a class of medications called nasal decongestants. it works by reducing swelling of the blood vessels in the nasal passages.about Phenylephrine'},
 {'doc_id': '1ee18da3-2fa4-4

In [ ]:
save_to = os.path.join(workspace_base_path, "datasets", "chunked_medicine_data.json")
out_path = Path(save_to)
out_path.parent.mkdir(parents=True, exist_ok=True)  # make folder if needed

with out_path.open("w", encoding="utf-8") as f:
    json.dump(approch_orig_data, f, ensure_ascii=False, indent=2)
